<a href="https://colab.research.google.com/github/raheeltahir55/CE888/blob/main/Project/Proj_MultiEval_Hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### HATE ####
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import io
import requests

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
np.random.seed(500)

In [ ]:
#import base64
#import requests

#master = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt"
#req = requests.get(master)
#req = req.text
#for line in req:
#   print(line)

In [4]:
# from google.colab import files
# uploaded= files.upload()
# file= "Train_Text.txt"
# uploaded[file].decode("utf-8").split("\r\n")

r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt', allow_redirects= True)
open('train_text.txt', 'wb').write(r.content)
example1 = "/content/train_text.txt"
with open(example1, "r") as file1:
    data = file1.readlines()

In [5]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt', allow_redirects= True)
open('train_labels.txt', 'wb').write(r.content)
example1 = "/content/train_labels.txt"
with open(example1, "r") as file1:
    data1 = file1.readlines()

In [6]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt', allow_redirects= True)
open('test_text.txt', 'wb').write(r.content)
example1 = "/content/test_text.txt"
with open(example1, "r") as file1:
    data2 = file1.readlines()

In [7]:
r= requests.get('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt', allow_redirects= True)
open('test_labels.txt', 'wb').write(r.content)
example1 = "/content/test_labels.txt"
with open(example1, "r") as file1:
    data3 = file1.readlines()

In [8]:
# data = uploaded[file].decode("utf-8").split("\r\n")
# data1 = uploaded1[file1].decode("utf-8").split("\r\n")
# data2 = uploaded2[file2].decode("utf-8").split("\r\n")
# data3 = uploaded3[file3].decode("utf-8").split("\r\n")

data= [data[i].strip() for i in range(len(data))]
data1= [data1[i].strip() for i in range(len(data1))]
data2= [data2[i].strip() for i in range(len(data2))]
data3= [data3[i].strip() for i in range(len(data3))]

In [9]:
### df dataframe is training data

df= pd.DataFrame()
df['Text']=range(0, len(data))

In [10]:
for i in range(len(data)):
  df.iloc[i, df.columns.get_loc('Text')]= data[i]
print(df.head())

                                                Text
0  @user nice new signage. Are you not concerned ...
1  A woman who you fucked multiple times saying y...
2  @user @user real talk do you have eyes or were...
3  your girlfriend lookin at me like a groupie in...
4                        Hysterical woman like @user


In [11]:
df['Label']=range(0, len(data))

In [12]:
for i in range(len(data1)):
  df.iloc[i, df.columns.get_loc('Label')]= data1[i]
print(df.head())

                                                Text Label
0  @user nice new signage. Are you not concerned ...     0
1  A woman who you fucked multiple times saying y...     1
2  @user @user real talk do you have eyes or were...     1
3  your girlfriend lookin at me like a groupie in...     1
4                        Hysterical woman like @user     0


In [13]:
# Step - a : Remove blank rows if any.
df['Text'].dropna(inplace=True)

In [14]:
# Step - b : Change all the text to lower case.
df['Text'] = [entry.lower() for entry in df['Text']]

In [15]:
df.head()

,Text,Label
0,@user nice new signage. are you not concerned ...,0
1,a woman who you fucked multiple times saying y...,1
2,@user @user real talk do you have eyes or were...,1
3,your girlfriend lookin at me like a groupie in...,1
4,hysterical woman like @user,0


In [16]:
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df['Text']= [word_tokenize(entry) for entry in df['Text']]

In [17]:
df.head()

,Text,Label
0,"[@, user, nice, new, signage, ., are, you, not...",0
1,"[a, woman, who, you, fucked, multiple, times, ...",1
2,"[@, user, @, user, real, talk, do, you, have, ...",1
3,"[your, girlfriend, lookin, at, me, like, a, gr...",1
4,"[hysterical, woman, like, @, user]",0


In [18]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
#df['Text']= [preprocess(entry) for entry in df['Text']]

In [ ]:
for entry in df['Text']:
  print(entry)

In [20]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [21]:
for index,entry in enumerate(df['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'Text_Final'] = str(Final_words) 

In [22]:
df.head()
#print(df['Text_Final'].dtypes)

,Text,Label,Text_Final
0,"[@, user, nice, new, signage, ., are, you, not...",0,"['user', 'nice', 'new', 'signage', 'concern', ..."
1,"[a, woman, who, you, fucked, multiple, times, ...",1,"['woman', 'fuck', 'multiple', 'time', 'say', '..."
2,"[@, user, @, user, real, talk, do, you, have, ...",1,"['user', 'user', 'real', 'talk', 'eye', 'gouge..."
3,"[your, girlfriend, lookin, at, me, like, a, gr...",1,"['girlfriend', 'lookin', 'like', 'groupie', 'b..."
4,"[hysterical, woman, like, @, user]",0,"['hysterical', 'woman', 'like', 'user']"


In [ ]:
#df['Text_Final']= df['Text_Final'].astype(str)

In [23]:
print(df.dtypes)

Text          object
Label         object
Text_Final    object
dtype: object


In [24]:
### df1 dataframe is test data

df1= pd.DataFrame()
df1['Text']=range(0, len(data2))
df1['Label']=range(0, len(data2))

In [25]:
for i in range(len(data2)):
  df1.iloc[i, df1.columns.get_loc('Text')]= data2[i]
print(df1.head())

                                                Text  Label
0  @user , you are correct that Reid certainly is...      0
1             Whoever just unfollowed me you a bitch      1
2  @user @user Those People Invaded Us!!! They DO...      2
3  stop JUDGING bitches by there cover, jus cuz s...      3
4  how about i knock heads off and send them gift...      4


In [26]:
for i in range(len(data3)):
  df1.iloc[i, df1.columns.get_loc('Label')]= data3[i]
print(df1.head())

                                                Text Label
0  @user , you are correct that Reid certainly is...     0
1             Whoever just unfollowed me you a bitch     1
2  @user @user Those People Invaded Us!!! They DO...     1
3  stop JUDGING bitches by there cover, jus cuz s...     1
4  how about i knock heads off and send them gift...     1


In [27]:
# Step - a : Remove blank rows if any.

#df1['Text']= df1['Text'].astype(str)

df1['Text'].dropna(inplace=True)

# Step - b : Change all the text to lower case.
df1['Text'] = [entry.lower() for entry in df1['Text']]

# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df1['Text']= [word_tokenize(entry) for entry in df1['Text']]

for index,entry in enumerate(df1['Text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df1.loc[index,'Text_Final'] = str(Final_words) 

In [28]:
df.head()

,Text,Label,Text_Final
0,"[@, user, nice, new, signage, ., are, you, not...",0,"['user', 'nice', 'new', 'signage', 'concern', ..."
1,"[a, woman, who, you, fucked, multiple, times, ...",1,"['woman', 'fuck', 'multiple', 'time', 'say', '..."
2,"[@, user, @, user, real, talk, do, you, have, ...",1,"['user', 'user', 'real', 'talk', 'eye', 'gouge..."
3,"[your, girlfriend, lookin, at, me, like, a, gr...",1,"['girlfriend', 'lookin', 'like', 'groupie', 'b..."
4,"[hysterical, woman, like, @, user]",0,"['hysterical', 'woman', 'like', 'user']"


In [29]:
df1.head()

,Text,Label,Text_Final
0,"[@, user, ,, you, are, correct, that, reid, ce...",0,"['user', 'correct', 'reid', 'certainly', 'weas..."
1,"[whoever, just, unfollowed, me, you, a, bitch]",1,"['whoever', 'unfollowed', 'bitch']"
2,"[@, user, @, user, those, people, invaded, us,...",1,"['user', 'user', 'people', 'invade', 'u', 'bel..."
3,"[stop, judging, bitches, by, there, cover, ,, ...",1,"['stop', 'judging', 'bitch', 'cover', 'jus', '..."
4,"[how, about, i, knock, heads, off, and, send, ...",1,"['knock', 'head', 'send', 'gift', 'wrap', 'mom..."


In [30]:
t_df= pd.concat([df, df1])

#Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect = TfidfVectorizer(max_features=6000)
Tfidf_vect.fit(t_df['Text_Final'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=6000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [31]:
t_df['Label'].value_counts()

0    6935
1    5035
Name: Label, dtype: int64

In [32]:
Train_X= df['Text_Final']
Train_Y= df['Label']

In [ ]:
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
print(Train_X_Tfidf)

In [35]:
# df1['Text_Final']= df1['Text_Final'].astype(str)

In [36]:
Test_X= df1['Text_Final']
Test_Y= df1['Label']

Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [41]:
print(Test_X[0])

['user', 'correct', 'reid', 'certainly', 'weasel', 'sadly', 'get', 'weasel', 'user', 'sen', 'mcconnell', 'user', 'corrupt', 'mueller', 'investigation', 'stop', 'maga', 'kag', 'potus', 'trump', 'news', 'votered', 'nodaca', 'usa']


In [40]:
print(Train_X_Tfidf[4,:])

  (0, 5889)	0.46093703537618935
  (0, 5617)	0.2704545435932169
  (0, 3138)	0.4977275351310296
  (0, 2477)	0.6831242127466892


In [39]:
print(Test_X_Tfidf[0,:])

  (0, 5776)	0.5245332284741525
  (0, 5703)	0.17062117197968327
  (0, 5617)	0.16695067668810973
  (0, 5611)	0.15011967841622545
  (0, 5498)	0.11631704220011843
  (0, 5137)	0.12776557001344474
  (0, 4780)	0.2557668983516498
  (0, 4644)	0.23864587334059517
  (0, 4399)	0.26226661423707626
  (0, 4087)	0.16681008163991645
  (0, 3613)	0.1307148844684054
  (0, 3581)	0.15689180203550454
  (0, 3498)	0.23280074584597032
  (0, 3339)	0.22012081955500767
  (0, 3252)	0.11597918306390662
  (0, 2888)	0.15374477432452438
  (0, 2730)	0.22577618769753582
  (0, 2114)	0.09903464305038297
  (0, 1205)	0.2139658172492953
  (0, 1200)	0.2139658172492953
  (0, 943)	0.2504562437888357


In [38]:
print(Test_X_Tfidf[2,:])

  (0, 5966)	0.2996560659095236
  (0, 5617)	0.20059381923158565
  (0, 4550)	0.23515538642922806
  (0, 3951)	0.21306982003758032
  (0, 3754)	0.2883321128418398
  (0, 3613)	0.23558393196797703
  (0, 2722)	0.35050206768008074
  (0, 2534)	0.2895207644595778
  (0, 2506)	0.38084915294360816
  (0, 2423)	0.2972802515761778
  (0, 991)	0.3370251647496475
  (0, 207)	0.2774072856361166


In [44]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  52.996632996633


In [45]:
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y, average='macro')*100)

Naive Bayes F1 Score ->  50.96879359287348


In [46]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  51.27946127946128


In [48]:
print("SVM F1 Score -> ",f1_score(predictions_SVM, Test_Y, average='macro')*100)

SVM F1 Score ->  47.97042917837131


In [49]:
print(predictions_NB[0:4])

['1' '1' '1' '0']


In [50]:
print(predictions_SVM[0:4])

['1' '1' '1' '1']


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {  
'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
}  
grid_1= GridSearchCV(Naive, parameters)
grid_1.fit(Train_X_Tfidf,Train_Y)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (1, 0.1, 0.01, 0.001, 0.0001, 1e-05)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
# print best parameter after tuning 
print(grid_1.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid_1.best_estimator_) 

{'alpha': 1}
MultinomialNB(alpha=1, class_prior=None, fit_prior=True)


In [51]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB(alpha=1, class_prior=None, fit_prior=True)
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes F1 Score -> ",f1_score(predictions_NB, Test_Y, average='macro')*100)

Naive Bayes Accuracy Score ->  52.996632996633
Naive Bayes F1 Score ->  50.96879359287348


In [ ]:
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVM, param_grid, refit = True, verbose = 3) 
 
# fitting the model for grid search 
grid.fit(Train_X_Tfidf,Train_Y) 

# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.617, total=   8.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.5s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.614, total=   8.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   17.0s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.612, total=   8.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.611, total=   8.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.618, total=   8.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.581, total=   8.1s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.582, total=   8.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.581, total=   8.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.581, total=   8.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 17.2min finished


{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [52]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

print("SVM F1 Score -> ",f1_score(predictions_SVM, Test_Y, average='macro')*100)

SVM Accuracy Score ->  51.07744107744108
SVM F1 Score ->  48.03177142799339
